In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import workbook, load_workbook
import os
import time
import pandas as pd
import numpy as np
import gc

# Created By Alexander Vu
# Ver. 1.2

# This program creates the company list for company ID listed below by the RA.
# This is done using the RA's UT EID login
# Please see tutorial on on how to use.

# ################################################################################################
# Fill in your variables

name = ""
UT_EID = ""
UT_Password = ""
driverDirectory = ""
workDirectory = r''
start = int()
end = int()

############################### Optional #########################################################
# Please email which company give errors before using.
# This way they are incorporated into the new version.
speed= int(1) # Default int(1) is the fastest. Increase the number to make it slower. 
              # Example: int(2) indicates 2 times slower
              # Example: int(4) indicates 4 times slower
        
minhsWaitTime = int(0) # Number of seconds to wait while loading Factiva website.

skip2 = []# List company number between 1-50 to skip in cell 2. Only use if you know they company gives an error.
          # Example: [2,34,48] will skip company number 2, 34, 48 in cell 2
          # Example: [23] will skip company number 23 in cell 2
        
haoKunsSkip = []# List company number between 1-50 to skip in cell 3. Only use if you know they company gives an error.
          # Example: [2,34,48] will skip company number 2, 34, 48 in cell 3
          # Example: [23] will skip company number 23 in cell 3
        
##################################################################################################
print("Welcome to to Alexander Vu's Factiva data scrapper!")

print("\nWe are now logging into your Factiva account!")

driver = webdriver.Edge(driverDirectory)
driver.get('https://guides.lib.utexas.edu/db/144')
time.sleep(int(1*speed))
try:
    username = driver.find_element_by_id("username")
    username.send_keys(UT_EID)
    password = driver.find_element_by_id("password")
    password.send_keys(UT_Password)
    password.send_keys(Keys.RETURN)
    print("As a victim of Duo, please approve access.")
    try:
        element = WebDriverWait(driver,60).until(
            EC.presence_of_element_located((By.ID,"trust-browser-button"))
        )
    finally:
        print()
    trustBrowser = driver.find_element_by_id("trust-browser-button")
    trustBrowser.send_keys(Keys.RETURN)    
except NoSuchElementException:
    print("It seems that you are already logged in.")

###============================================================================
########## RETRIEVE PART OF DATA NEED TO WORK ON (CREDIT: ALEXANDER TANG)
###============================================================================

print("Retrieving your assignment!")

alldata = pd.read_excel(workDirectory + '\\STEP4_Factiva_company_list.xlsx')
#mydata = alldata[(alldata["id"] >=...) & (alldata["id"] <=...)]   ## replace the three dots by your numbers
#example:
mydata = alldata[(alldata["id"] >=start) & (alldata["id"] <=end)]      ##get companies ID from 1 to 20
mydata  ## print data to check

mydata.to_excel(workDirectory + '\\' + 'STEP4_Factiva_company_list_No'+str(start)+'_'+str(end)+'_'+name + '.xlsx')   ## replace the number by your numbers
###============================================================================


print("Done!")
wb = load_workbook(workDirectory + '\\' + 'STEP4_Factiva_company_list_No'+str(start)+'_'+str(end)+'_'+name + '.xlsx')
ws = wb.active
time.sleep(minhsWaitTime)
try:
    element = WebDriverWait(driver,60).until(
        EC.presence_of_element_located((By.ID,"isrd"))
    )
finally:
    print("\nFactiva page has finished loading.")
        
english = driver.find_element_by_class_name("filterPillWrap")
english.click()
duplicates = driver.find_element_by_id("isrd")
duplicates.click()
duplicates.send_keys(Keys.UP+Keys.RETURN)
companyTab = driver.find_element_by_xpath("//a[@class='fesTabLinkFix'][normalize-space()='Company']")
companyTab.click()


#driver.quit()


Welcome to to Alexander Vu's Factiva data scrapper!

We are now logging into your Factiva account!
As a victim of Duo, please approve access.

Retrieving your assignment!
Done!

Factiva page has finished loading.


In [2]:
#Cell 2 Collects Name, Code, Primary Industry: Company Flag, Year started, Address, Country Code, Location Type, Global Ultimate, Domestic Ultimate, and Immediate Parent

companydf = pd.read_excel(workDirectory + '\\' + 'STEP4_Factiva_company_list_No'+str(start)+'_'+str(end)+'_'+name + '.xlsx',index_col=None, na_values=['NA'],usecols="D")
companyName = companydf.values.tolist()
countryNamedf = pd.read_excel(workDirectory + '\\' + 'STEP4_Company_frequency.xlsx',sheet_name="country_code",index_col=None, na_values=['NA'],usecols="A")
countryName = countryNamedf.values.tolist()
countryAbbredf = pd.read_excel(workDirectory + '\\' + 'STEP4_Company_frequency.xlsx',sheet_name="country_code",index_col=None, na_values=['NA'],usecols="B")
countryAbbre = countryAbbredf.values.tolist()
countryDic={}
#Create country Code mapping
for h in range(0,len(countryName)):
    countryDic[str(countryName[h][0])]=str(countryAbbre[h][0])

class company_Info:
    def __init__(self, name, infoTxt):
        self.__comName = name
        self.__lstOfrawtxt = infoTxt.split("\n")
        #print(self.__lstOfrawtxt)
        self.__lstOftxt = []
        for item in self.__lstOfrawtxt:
            if("Web Address:" not in item.strip()):
                self.__lstOftxt.append(item.strip())
    def getName(self):
        return str(self.__comName)
    def getCode(self):
        for line in self.__lstOftxt:
            if "Code:" in line:
                return line[line.find(":")+1:].strip()
            
    def getPrimaryIndustry(self):
        for line in self.__lstOftxt:
            if "Primary Industry:" in line:
                return line[line.find(":")+1:].strip()+""
    def getYearStarted(self):
        for line in self.__lstOftxt:
            if "Year Started:" in line:
                return line[line.find(":")+1:].strip()+""

    def getAddress(self):
        self.__fullAddress = ""
        for l in range(1,len(self.__lstOftxt)):
            if ("Address:" in self.__lstOftxt[l]):
                self.__fullAddress += self.__lstOftxt[l][self.__lstOftxt[l].find(":")+1:].strip()+" "
                for j in range(l+1,len(self.__lstOftxt)):
                    if ":" not in self.__lstOftxt[j]:
                        self.__fullAddress += self.__lstOftxt[j].strip()+" "
            elif ("Location:" in self.__lstOftxt[l]):
                self.__fullAddress += self.__lstOftxt[l][self.__lstOftxt[l].find(":")+1:].strip()+" "
                for j in range(l+1,len(self.__lstOftxt)):
                    if ":" not in self.__lstOftxt[j]:
                        self.__fullAddress += self.__lstOftxt[j].strip()+" "
            
        return self.__fullAddress+""
    def getCountryCode(self):
        for key in countryDic:
            if key in self.getAddress():
                return countryDic[key]
        return
         
    def getLocationType(self):
        for line in self.__lstOftxt:
            if "Location Type:" in line:
                return line[line.find(":")+1:].strip()+""
    def getGlobUlt(self):
        for line in self.__lstOftxt:
            if "Global Ultimate:" in line:
                return line[line.find(":")+1:].strip()+""
    def getDomUlt(self):
        for line in self.__lstOftxt:
            if "Domestic Ultimate:" in line:
                return line[line.find(":")+1:].strip()+""
    def getImPar(self):
        for line in self.__lstOftxt:
            if "Immediate Parent:" in line:
                return line[line.find(":")+1:].strip()+""
    def getCompanyFlag(self):
        if(self.getPrimaryIndustry()!=None):
            return 1
        else:
            return 0
        
    def __str__(self):
        print("Name: " + str(self.getName()))
        print("Code: " + str(self.getCode()))
        print("Primary Industry: " + str(self.getPrimaryIndustry()))
        print("Company Flag: " +str(self.getCompanyFlag()))
        print("Year Started: " + str(self.getYearStarted()))
        print("Address: " + str(self.getAddress()))
        print("Country Code: "+str(self.getCountryCode()))
        print("Locations Type: " + str(self.getLocationType()))
        print("Global Ultimate: " + str(self.getGlobUlt()))
        print("Domestic Ultimate: " + str(self.getDomUlt()))
        print("Immediate Parent: " + str(self.getImPar()))
        return""
    def writeToExcel(self,posNum):
        rowNum = str(posNum+2)
        if(self.getCompanyFlag()==None):
            ws["C"+rowNum].value=""
        else:
            ws["C"+rowNum].value=str(self.getCompanyFlag())
        if(self.getCode()==None):
            ws["F"+rowNum].value=""
        else:
            ws["F"+rowNum].value=str(self.getCode())
            
        if(self.getCountryCode()==None):
            ws["G"+rowNum].value=""
        else:
            ws["G"+rowNum].value=str(self.getCountryCode())
            
        if(self.getPrimaryIndustry()==None):
            ws["H"+rowNum].value=""
        else:
            ws["H"+rowNum].value=str(self.getPrimaryIndustry())
            
        if(self.getAddress()==None):
            ws["J"+rowNum].value=""
        else:
            ws["J"+rowNum].value=str(self.getAddress())
            
        if(self.getLocationType()==None):
            ws["K"+rowNum].value=""
        else:
            ws["K"+rowNum].value=str(self.getLocationType())
            
        if(self.getYearStarted()==None):
            ws["L"+rowNum].value=""
        else:
            ws["L"+rowNum].value=str(self.getYearStarted())
            
        if(self.getGlobUlt()==None):
            ws["M"+rowNum].value=""
        else:
            ws["M"+rowNum].value=str(self.getGlobUlt())
            
        if(self.getDomUlt()==None):
            ws["N"+rowNum].value=""
        else:
            ws["N"+rowNum].value=str(self.getDomUlt())
            
        if(self.getImPar()==None):
            ws["O"+rowNum].value=""
        else:
            ws["O"+rowNum].value=str(self.getImPar())
        
        print("Information recorded in Excel!")
        return 


    
#Modify the list into list of strings
for i in range(0,len(companyName)):
    companyNamestr = str(companyName[i][0])
    search = driver.find_element_by_xpath("//input[@id='coTxt']")
    search.clear()
    print("Company "+str(i+1)+": "+companyNamestr+"\n")
    if (i+1) not in skip2:
        search.send_keys(str(companyNamestr)+Keys.RETURN)
        CssSelectorStr = "a[title=\'Get details for "+companyNamestr+" \']"
        XPATHstr = "//a[@title=\'Get details for "+companyNamestr+"\']//img"
        time.sleep(int(5*speed))
        try:
            #driver.find_element(By.CSS_SELECTOR, CssSelectorStr)
            try:
                result = driver.find_element(By.CSS_SELECTOR, CssSelectorStr).click()
            except (NoSuchElementException or ElementNotInteractableException):
                try:
                    result = driver.find_element(By.XPATH, XPATHstr).click()
                except (NoSuchElementException or ElementNotInteractableException):
                    print("")
            time.sleep(int(5*speed))
            table = driver.find_element(By.XPATH,"//table[@class='infoTable']")
    #         print("Company #"+str(i+1))
    #         print(table.text)
            companyObj = company_Info(str(companyNamestr),str(table.text))
            print(companyObj)
            companyObj.writeToExcel(i)
            #print("-----------------------------------------------------------------------")

            actions = ActionChains(driver)
            actions.send_keys(Keys.ESCAPE)

            actions.perform()

        except (NoSuchElementException or ElementNotInteractableException):
            try: #Try adding a period to the end of company name?!
                companyNamestrp = companyNamestr+"."
                CssSelectorStrp = "a[title=\'Get details for "+companyNamestrp+" \']"
                XPATHstrp = "//a[@title=\'Get details for "+companyNamestrp+"\']//img"
                search.send_keys("."+Keys.RETURN)
                time.sleep(int(5*speed))
                #driver.find_element(XPATHstrp)
                try:
                    resultp = driver.find_element(By.CSS_SELECTOR, CssSelectorStrp).click()
                except (NoSuchElementException or ElementNotInteractableException):
                    try:
                        resultp = driver.find_element(By.XPATH, XPATHstrp).click()
                    except (NoSuchElementException or ElementNotInteractableException):
                        print("")
                time.sleep(int(5*speed))
                tablep = driver.find_element(By.XPATH,"//table[@class='infoTable']")
    #             print("Company #"+str(i+1))
    #             print(tablep.text)
                companyObjp = company_Info(str(companyNamestrp),str(tablep.text))
                print(companyObjp)
                companyObjp.writeToExcel(i)
                #print("-----------------------------------------------------------------------")

                actions = ActionChains(driver)
                actions.send_keys(Keys.ESCAPE)

                actions.perform()

            except (NoSuchElementException or ElementNotInteractableException):
                print("Company #"+str(i+1))
                print("Company information cannot be found for "+str(companyName[i][0]))
                #print("-----------------------------------------------------------------------")
    
    else:
        print("Skipping "+companyNamestr)
                    
    print("-----------------------------------------------------------------------")
    search.clear()

wb.save(workDirectory + '\\' + 'STEP4_Factiva_company_list_No'+str(start)+'_'+str(end)+'_'+name + '.xlsx')
print("\nExcel Sheet saved!")


Company 1: BB Biotech AG

Name: BB Biotech AG
Code: BBBTEC
Primary Industry: Biotechnology Services
Company Flag: 1
Year Started: 1993
Address: Schwertstrasse 6 Schaffhausen, Zurich 8200 Switzerland 
Country Code: CH
Locations Type: None
Global Ultimate: BB Biotech AG
Domestic Ultimate: BB Biotech AG
Immediate Parent: BB Biotech AG

Information recorded in Excel!
-----------------------------------------------------------------------
Company 2: Taubman Centers Inc



KeyboardInterrupt: 

In [ ]:
# Cell 3 collects SIC, NACE, NAICS information

listOfCssSelector = ["body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)","body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(3) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)","body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)","“body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(3) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)"]
def getTable(cN):
    try:#9
        DSNN = driver.find_element(By.CSS_SELECTOR, "body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(3) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)")
        comSNNObj = Company_SNN(DSNN.text)
        comSNNObj.writeToExcelSNN(i)
        print(comSNNObj)
    except NoSuchElementException:
        try:#8
            DSNN = driver.find_element(By.CSS_SELECTOR, "body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)")
            comSNNObj = Company_SNN(DSNN.text)
            comSNNObj.writeToExcelSNN(i)
            print(comSNNObj) 
        except NoSuchElementException:                  
            try:#7
                DSNN = driver.find_element(By.CSS_SELECTOR, "body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)")
                comSNNObj = Company_SNN(DSNN.text)
                comSNNObj.writeToExcelSNN(i)
                print(comSNNObj) 
            except NoSuchElementException:
                try:#6
                    DSNN = driver.find_element(By.CSS_SELECTOR, "body > form:nth-child(10) > div:nth-child(7) > table:nth-child(4) > tbody:nth-child(1) > tr:nth-child(1) > td:nth-child(2) > div:nth-child(1) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(3) > td:nth-child(1) > div:nth-child(1) > div:nth-child(1) > table:nth-child(2) > tbody:nth-child(1)")
                    comSNNObj = Company_SNN(DSNN.text)
                    comSNNObj.writeToExcelSNN(i)
                    print(comSNNObj) 
                except NoSuchElementException:
                    print("Cannot find the table. Please collect data manually.")


comMkt = driver.find_element(By.LINK_TEXT,"Companies/Markets")
comMkt.click()
lookUp = driver.find_element(By.ID,"ct")

class Company_SNN:
    def __init__(self,text):
        self.__lstTxt=text.split("\n")
        for g in range(0,len(self.__lstTxt)):
            self.__lstTxt[g]=self.__lstTxt[g][self.__lstTxt[g].find(" "):].strip()
        self.__SIC= self.__lstTxt[1][0:self.__lstTxt[1].find(" ")].strip()
        self.__SICdes=self.__lstTxt[1][self.__lstTxt[1].find(" "):].strip()
        if("N/A" not in self.__lstTxt[2]): 
            self.__NACE=self.__lstTxt[2][0:self.__lstTxt[2].find(" ")].strip()
            self.__NACEdes=self.__lstTxt[2][self.__lstTxt[2].find(" "):].strip()
        else:#6
            self.__NACE=""
            self.__NACEdes=""
        self.__NAICS=self.__lstTxt[3][0:self.__lstTxt[3].find(" ")].strip()
        self.__NAICSdes=self.__lstTxt[3][self.__lstTxt[3].find(" "):].strip()
    def getSIC(self):
        return self.__SIC
    def getSICdes(self):
        return self.__SICdes
    def getNACE(self):
        return self.__NACE
    def getNACEdes(self):
        return self.__NACEdes
    def getNAICS(self):
        return self.__NAICS
    def getNAICSdes(self):
        return self.__NAICSdes
    def __str__(self):
        print("SIC: "+self.getSIC())
        print("SIC Description: "+self.getSICdes())
        print("NACE: "+self.getNACE())
        print("NACE Description: "+self.getNACEdes())
        print("NAICS: "+self.getNAICS())
        print("NAICS Description: "+self.getNAICSdes())
        return ""
    def writeToExcelSNN(self,posNum):
        rowNum=str(posNum+2)
        ws["P"+rowNum].value=self.getNAICS()
        ws["Q"+rowNum].value=self.getNAICSdes()
        ws["R"+rowNum].value=self.getNACE()
        ws["S"+rowNum].value=self.getNACEdes()
        ws["T"+rowNum].value=self.getSIC()
        ws["U"+rowNum].value=self.getSICdes()
        print("\nInformation recorded in Excel!")
        return
    
for i in range(0,len(companyName)):
    comName = companyName[i][0]
    print("-----------------------------------------------------------------------")
    print("Company " + str(i+1) + ": "+comName)
    print("")
    
#Check what page we are on   
    if (i+1) not in haoKunsSkip:
        lookUp.send_keys(comName+Keys.RETURN)
        time.sleep(int(3*speed))
        # Check if it goes directly to the page
        try:#5
            currentPage = driver.find_element(By.CSS_SELECTOR, ".breadcrumbs")
            if("Snapshot" in currentPage.text): # Pages shows up immediately
                path1 = True
                #print("The page showed up immediately!")
            else:#5
                path1 = False
        except NoSuchElementException:
            path1 = False
            
        # check if there is no results
        try:#4
            currentPage2 = driver.find_element(By.ID, "contentBottom")
            if("No results" in currentPage2.text):
                noResults = True
            else:#4
                noResults = False
        except NoSuchElementException:
            noResults = False
            
# Page showed up immediately           
        if(path1==True):
            getTable(i) 
# No results
        elif(noResults==True):#2
            print("No results showed up.")      
# There are options
        else:#3
            print("Looks like there's options...\n")
        
            try:#3
                #Select correct option
                print("Searching for correct option...\n")
                companyXPATHSNN = "//a[normalize-space()='"+comName+"']"
                #print(companyXPATHSNN)
                selectCompany = driver.find_element(By.XPATH,companyXPATHSNN)
                selectCompany.click()
                time.sleep(int(3*speed))
                getTable(i)
#Try with period at end
            except NoSuchElementException:
                print("Let's try adding period at the end of the company name...\n")

                try:#2
                    print("Searching for correct option again...\n")
                    companyXPATHSNN = "//a[normalize-space()='"+comName+".']"
                    #print(companyXPATHSNN)
                    selectCompany = driver.find_element(By.XPATH,companyXPATHSNN)
                    selectCompany.click()
                    time.sleep(int(3*speed))
                    getTable(i)

                except NoSuchElementException:
                    try:
                        print("Let's try a case sensitive option.\n")
                        comNameCase = comName.lower().title()
                        #print(comNameCase)
                        companyXPATHSNN = "//a[normalize-space()='"+comNameCase+"']"
                        selectCompany = driver.find_element(By.XPATH,companyXPATHSNN)
                        selectCompany.click()
                        time.sleep(int(3*speed))
                        getTable(i)
                    except NoSuchElementException:
                        print("Cannot find correct option! Please collect information for "+str(comName)+" manually!")
       
    else:#1
        print("Skipping Company #"+str(i+1))
                    
                    
    try:#1
        lookUp = driver.find_element(By.ID,"comp_ct")
    except NoSuchElementException:
        print("It seems like there is an error... Please find "+comName+" manually!")
        comMkt = driver.find_element(By.LINK_TEXT,"Companies/Markets")
        comMkt.click()
        lookUp = driver.find_element(By.ID,"ct")
        
    time.sleep(int(5*speed))

wb.save(workDirectory + '\\' + 'STEP4_Factiva_company_list_No'+str(start)+'_'+str(end)+'_'+name + '.xlsx')
print("\nExcel Sheet saved!")

driver.quit()

print("\n                     _________   ")     
print("            (__)    /         \  ")     
print("            (oo)   ( All Done! ) ")   
print("     /-------\/  --'\_________/  ")    
print("    / |     ||                  ")  
print("   *  ||----||                  ")  
print("      ~~    ~~                  ")     